# Load eigen vector and eigen values

In [61]:
session_name = '20230516'
region_name = 'Interposed nucleus' # Lobules IV-V  Interposed nucleus
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
save_path = f'/home/zhangyuhao/Desktop/Result/Online_Move_Correct/Spectrum_analysis/{session_name}'
evalues = np.load(save_path+f'/{region_name}_evalues.npy')  # (num_trials, 4, K)
evectors = np.load(save_path+f'/{region_name}_evectors.npy') # (num_trials, 4, K, K)
mags = np.load(save_path+f'/{region_name}_mags.npy')  # (num_trials, 4)

# Refine magnitude in different stage

In [ ]:
# Slice the 'mags' numpy array along the second dimension
sliced_mags = [mags[:, i, :] for i in range(4)]
stages = ['stage1','stage2','stage3','stage4']
# Convert the 2D ndarrays into dataframes
dfs = []
for i, mag_2d in enumerate(sliced_mags):
    df = pd.DataFrame({
        'mags': mag_2d.flatten(),
        'stage': stages[i]
    })
    dfs.append(df)

# Concatenate all dataframes into a single dataframe
result_df = pd.concat(dfs, ignore_index=True)
# Filter out rows where 'mags' column values are less than 0.02
result_df = result_df[result_df['mags'] >= 0.02]
result_df

# Find Non-common eigen vector

In [63]:
# 假设 eigenvalue 和 eigenvector 是已经加载的numpy数组
threshold = 0.99 # 相似度阈值，可根据需要调整

# 初始化存储结果
feature_vectors = []  #存储所有trial最大特征值的特征向量
stage0_exist = []
stage1_exist = []
stage3_exist = []

for trial in range(evectors.shape[0]):
    # 获取第三阶段（索引2）的特征值
    stage2_evals = evalues[trial, 2, :]
    max_idx = np.argmax(stage2_evals)
    if stage2_evals[max_idx] >= 1:  # 仅考虑特征值大于1的情况
        # 提取对应的特征向量
        current_vec = evectors[trial, 2, :, max_idx]
        #current_vec = current_vec / np.linalg.norm(current_vec) 
        
        # 检查阶段0是否存在相似向量
        stage0_vecs = evectors[trial, 0, :, :]  # 形状(440, 440)
        dots_stage0 = np.abs(np.dot(stage0_vecs.T, current_vec))
        stage0_exist.append(np.max(dots_stage0) >= threshold)
        
        # 检查阶段1是否存在相似向量
        stage1_vecs = evectors[trial, 1, :, :]
        dots_stage1 = np.abs(np.dot(stage1_vecs.T, current_vec))
        stage1_exist.append(np.max(dots_stage1) >= threshold)

        # 检查阶段4是否存在相似向量
        stage3_vecs = evectors[trial, 3, :, :]  # 形状(440, 440)
        dots_stage3 = np.abs(np.dot(stage3_vecs.T, current_vec))
        stage3_exist.append(np.max(dots_stage3) >= threshold)
        
        feature_vectors.append(current_vec)

# 转换为numpy数组
feature_vectors = np.array(feature_vectors)

# 计算相似度矩阵
similarity_matrix = np.abs(np.dot(feature_vectors, feature_vectors.T))

matching_pairs = []
for i in range(feature_vectors.shape[0]):
    for j in range(i + 1, feature_vectors.shape[0]):  # 仅遍历上三角，避免重复
        if similarity_matrix[i, j] >= threshold:
            matching_pairs.append((i+1, j+1))

# 输出结果
if len(matching_pairs) > 0:
    print("满足条件的trial对:")
    for pair in matching_pairs:
        print(f"Trial {pair[0]} 与 Trial {pair[1]}")
else:
    print("没有满足条件的trial对。")

# 输出结果
print("各trial中第三阶段的最大特征向量是否存在于阶段1：", stage0_exist)
print("各trial中第三阶段的最大特征向量是否存在于阶段2：", stage1_exist)
print("各trial中第三阶段的最大特征向量是否存在于阶段4：", stage3_exist)

满足条件的trial对:
Trial 3 与 Trial 4
Trial 3 与 Trial 7
Trial 3 与 Trial 8
Trial 4 与 Trial 7
Trial 4 与 Trial 8
Trial 6 与 Trial 12
Trial 6 与 Trial 13
Trial 7 与 Trial 8
Trial 12 与 Trial 13
各trial中第三阶段的最大特征向量是否存在于阶段1： [True, True, False, True, False, False, True, True, True, False, False, True, True]
各trial中第三阶段的最大特征向量是否存在于阶段2： [False, True, True, True, False, True, True, True, False, False, False, True, True]
各trial中第三阶段的最大特征向量是否存在于阶段4： [True, False, True, True, False, True, True, True, True, False, False, True, True]
